In [1]:
from tqdm import tqdm
import pandas as pd

# Load nusacrowd dataset

In [2]:
from nusacrowd import NusantaraConfigHelper
conhelps = NusantaraConfigHelper()
tydiqa_id_dset = conhelps.filtered(lambda x: "tydiqa_id" in x.dataset_name and "tydiqa_id_source" in x.config.name)[0].load_dataset()
example = tydiqa_id_dset["train"][0]
print(example)

/home/emmanueldave/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 3/3 [00:00<00:00, 554.44it/s]

{'id': 'indonesian-3276942661834849618-0', 'context': '"Hotel California" adalah judul lagu dari album dengan nama yang sama karya Eagles dan dirilis sebagai singel pada bulan Februari 1977. Kredit penulisan untuk lagu dibagi oleh Don Felder, Don Henley dan Glenn Frey. Perekaman asli Eagles dari lagu ini menampilkan Henley menyanyikan vokal dan diakhiri dengan bagian diperpanjang dari interaksi gitar listrik antara Felder dan Joe Walsh.', 'question': 'Siapakah vokali Hotel California?', 'label': 'Henley'}


# Test Created IndoPromptSource Template

In [3]:
from promptsource.templates import DatasetTemplates
tydiqa_id_prompts = DatasetTemplates("tydiqa_id", subset_name="tydiqa_id_source")
prompt = tydiqa_id_prompts["extract_answer"]
result = prompt.apply(example)
print("INPUT")
print(result[0])

print("\n")
print("OUTPUT")
print(result[1])

INPUT
Berdasarkan paragraf di bawah ini:
"Hotel California" adalah judul lagu dari album dengan nama yang sama karya Eagles dan dirilis sebagai singel pada bulan Februari 1977. Kredit penulisan untuk lagu dibagi oleh Don Felder, Don Henley dan Glenn Frey. Perekaman asli Eagles dari lagu ini menampilkan Henley menyanyikan vokal dan diakhiri dengan bagian diperpanjang dari interaksi gitar listrik antara Felder dan Joe Walsh.

Apakah jawaban dari pertanyaan: 
Siapakah vokali Hotel California?

Jawaban:


OUTPUT
Henley


# Full Pipeline

In [4]:
from promptsource.templates import TemplateCollection
collection = TemplateCollection()

In [5]:
nusa_templates = []
for k in collection.datasets_templates:
    if k is None or k[1] is None:
        continue
    
    if "nusa" in k[1]:
        nusa_templates.append(k)

print(len(nusa_templates))

82


In [6]:
import logging
# Set up the logger
logging.basicConfig(
    level=logging.DEBUG,  # Set the desired logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s [%(levelname)s]: %(message)s',  # Customize the log message format
    datefmt='%Y-%m-%d %H:%M:%S'  # Customize the date/time format
)

# Create a file handler to write logs into a file
file_handler = logging.FileHandler('app.log')
file_handler.setLevel(logging.DEBUG)  # Set the log level for the file handler

# Create a formatter for the file handler (customize the log format for the file)
file_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
file_handler.setFormatter(file_formatter)

logger = logging.getLogger("IndoP3 Dataset Generation")
logger.addHandler(file_handler)

In [7]:
all_data = []
checkpoint_save_path = "/home/emmanueldave/indo-t0/indo_promptsource/dataset/checkpoints"

for nusa_template_i in tqdm(nusa_templates):
    dataset_name = nusa_template_i[0]
    subset_name = nusa_template_i[1]

    # Load dataset
    dset = conhelps.filtered(lambda x: dataset_name in x.dataset_name and subset_name in x.config.name)[0].load_dataset()

    # Load prompt
    prompt = DatasetTemplates(dataset_name, subset_name=subset_name)

    # Iterate to each prompt templates
    for prompt_id in prompt.templates:
        template_name = prompt.templates[prompt_id].name

        for dataset_key in dset.keys():
            for example in dset[dataset_key]:
                data_details = {
                    "dataset_name": dataset_name,
                    "subset_name": subset_name,
                    "prompt_id": prompt_id,
                    "template_name": template_name,
                    "dataset_key": dataset_key,
                }
                input = None
                output = None

                try:
                    render = prompt[template_name].apply(example)
                    if len(render) != 2:
                        if len(render) == 1:
                            input = render[0]

                        logger.info(f"Output not available for {data_details}.")
                        break
                    else:
                        input = render[0]
                        output = render[1]
                except Exception as e:
                    logger.error(f"Exception occurred on {data_details}. Please rectify: {e}")
                    break
                
                data_details["input"] = input
                data_details["output"] = output

                all_data.append(
                    data_details
                )
        
    df_ = pd.DataFrame(all_data)
    print(f"df_.shape: {df_.shape}")
    df_.to_csv(f"{checkpoint_save_path}/ckpt.csv")

  1%|          | 1/82 [00:33<45:39, 33.82s/it]

df_.shape: (10000, 7)


100%|██████████| 3/3 [00:00<00:00, 359.27it/s]


df_.shape: (35000, 7)


100%|██████████| 3/3 [00:00<00:00, 111.15it/s]


...

 50%|█████     | 41/82 [2:20:40<15:21:51, 1349.07s/it]

Dataset xl_sum downloaded and prepared to /home/emmanueldave/.cache/huggingface/datasets/xl_sum/xl_sum_nusantara_t2t/1.0.0/370168f70442e4e1ff43cad9e260fa9f2cb0489c3458f64dfee803f69413749c. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 204.18it/s]
INFO:IndoP3 Dataset Generation:Output not available for {'dataset_name': 'xl_sum', 'subset_name': 'xl_sum_nusantara_t2t', 'prompt_id': '1b6038ac-04e4-4739-8b18-232a16ee6d9d', 'template_name': 'xl_sum_template_28', 'dataset_key': 'train'}.
INFO:IndoP3 Dataset Generation:Output not available for {'dataset_name': 'xl_sum', 'subset_name': 'xl_sum_nusantara_t2t', 'prompt_id': '1b6038ac-04e4-4739-8b18-232a16ee6d9d', 'template_name': 'xl_sum_template_28', 'dataset_key': 'validation'}.
INFO:IndoP3 Dataset Generation:Output not available for {'dataset_name': 'xl_sum', 'subset_name': 'xl_sum_nusantara_t2t', 'prompt_id': '1b6038ac-04e4-4739-8b18-232a16ee6d9d', 'template_name': 'xl_sum_template_28', 'dataset_key': 'test'}.


: 

: 

In [76]:
import pandas as pd
df = pd.DataFrame(all_data)
print(f"df.shape: {df.shape}")

df.shape: (55000, 7)


In [78]:
df.to_csv("first_run.csv")